# 00 Quick Example
In this brief notebook we investigate creating a simple hello world plugin for [KAGE]("https://github.com/thisismeamir/kage"). This module uses the help of the development kit for python available at [Kage-Python]("https://github.com/thisismeamir/kage-python").

## Note:
Throughout these example and in documentation we might mention the term "plugin" which is equivalent to "Node" in the Kage project. We use plugin here because it's mode understandable in the context of another project and not inside the Kage itself.


## Setting Up
To begin creating a plugin (node) in python. We first need to understand what a node is. Kage defines nodes as an abstraction of a task or a job. Given an input (json) it returns and output (json) and artifacts (side effects, files, and other object). A node can be defined in Kage using a node schema file which is a json with the following properties.

```json
{
  "name": "string",
  "version": "string",
  "type": "node",
  "model": {
    "execution_model": {
      "language": "string",
      "input_schema":  {},
      "output_schema": {},
      "artifacts": ["string"]
    },
    "source": "string",
    "working_directory": "string",
    "entry_file": "string",
    "output_directory": "string"
  },
  "metadata": {
    "description": "string",
    "authors": [
      {
        "name": "string",
        "email": "string",
        "url": "string"
      }
    ],
    "manual": "string",
    "repository": "string"
  }
}
```

To dive deep into understanding this file or nodes in general refer to the documentation of Kage itself. Here what we are interested in is the model key:
```json
  "model": {
    "execution_model": {
      "language": "string",
      "input_schema":  {},
      "output_schema": {},
      "artifacts": ["string"]
    },
    "source": "string",
    "working_directory": "string",
    "entry_file": "string",
  },
```
- `model`: Describes necessary information to Kage about the node (plugin).
- `execution_model`: Information about what language the plugin is written in. Helps because Kage commits to be language-agnostic, and therefore, needs to know what language this plugin/node is written in.
- `language`: Correct name of the language (e.g `python` not `py`).
- `input_schema`: An object that defines what does the json input should look like, both structure and type. This would be used further to make sure that the plugin would run correctly (apart from the errors that are internally caused by the plugin itself).
- `output_schema`: An object that shows what does the output of this plugin would look like. This is being used in graphs (another term for automation modules in Kage) to understand what is being sent to the next step of the process by this task.
- `artifacts`: Least of possible side effects and artifacts files.
- `source`: The path to the source of the plugin, or where the executables live.
- `working_directory`: Can be empty or in the input schema but if mentioned here tells Kage to execute this task in a specific folder.
- `entry_file`: The file that Kage would recognize as something that can be run for the desired output, defining this would enable you to have multiple files, folders and packages in the source directory, which makes writing more complex plugins easier.

## Writing a Plugin

### Step-01: Introducing Schemas To The Project
For any Kage-Python Plugin that we would write we have to define the two schemas, namely `input_schema` and `output_schema`

In [1]:
# Input schema
input_schema = {
    "type": "object",
    "required": ["user"],
        "user": {
            "type": "object",
            "properties": {
                "name": "string",
                "age": "integer",
                "email": "string"
            }
        }
}
# Output schema (optional)
output_schema = {
    "type": "object",
    "properties": {
        "greeting": "string",
        "user_summary": "string"
    }
}

Notice how we mentioned `type` key and value wherever we wanted to define an object. Otherwise just define the key followed by the type of the key as value.

### Step-02: Creating Functionality
This step is generally where you write your plugin application, any functionality that you have in mind can be written. You can also use multiple files, an already written project or anything you desire. For this notebook we would define two function in our project:

In [2]:
def create_greeting(name: str) -> str:
    return f"Hello, {name}!, This is a Kage Plugin"

def summarize_user(name: str, age: int, email: str) -> str:
    return f"{name} is {age} years old and can be reached at {email}"

### Step-03: Initializing Kage and Binding Functions
To initialize the plugin we would write something like the following in our `entry_file`. The general steps are:
1. **Making a Kage Object**: Give the Kage object input_schema and output_schema. And also passing the input_data to be evaluated.
2. **Binding Functions** to the object with a dictionary that tells Kage what which inputs go for which arguments.
3. **Executing Kage**: Execution of the plugin.

In [3]:
from kage.core import Kage
input_data = {
    "user": {
        "name": "Kid A",
        "age": 22,
        "email": "thisismeamir@outlook.com"
    }
}

# Initialize Kage
kage = Kage(input_schema, output_schema)

# Bind functions
kage.bind_function(
    create_greeting,
    {"name": "user.name"},
    output_key="greeting"
).bind_function(
    summarize_user,
    {"name": "user.name", "age": "user.age", "email": "user.email"},
    output_key="user_summary"
)

# Execute and get results
result = kage.execute(input_data)
print(kage.to_json())

{
  "greeting": "Hello, Kid A!, This is a Kage Plugin",
  "user_summary": "Kid A is 22 years old and can be reached at thisismeamir@outlook.com"
}
